# Senate vote percentages for all fifty states

In this notebook, we compute the two-party vote percentages for all fifty states using the turnout-weighted average of recent Senate election results.

## Data collection

Our data comes from election results spreadsheets made by the Federal Election Comission. For each federal election from 2002 to 2016, we have extracted the General Election columns from the "Senate By Party" sheet and included them as `.csv` files in the `raw_csv` folder of this repository. The entire summary documents may be downloaded from [this page](https://transition.fec.gov/pubrec/electionresults.shtml) on the FEC websie, which we accessed October 17th, 2018.

In addition to copying the columns, we removed all asterisks (for footnotes) from the state abbreviations, and normalized the column headers as follows:

* `State`: 2-letter state abbreviation
* `Democratic`: Democratic vote count
* `Republican`: Republican vote count
* `Other`: total non-Democratic, non-Republican vote count

### Additional notes on the data

We include these notes from the original FEC documents.

* "For states that had votes for both full and unexpired terms, the votes for both terms are included in the totals." This affects the following Senate totals:
    * 2010: Illinois, New York
    * 2014: Hawaii, Oklahoma, South Carolina
* "In some states, there were unopposed candidates whose names did not appear on a ballot and therefore received no votes."

## Data tidying

Now that we have the raw election results, we join them into a single [tidy](http://r4ds.had.co.nz/tidy-data.html) dataset covering all Senate election races in all states from 2002 to 2016.

We want each row of our table to be a record of a single Senate race. To do this, we join the contents of all of our raw CSV files into a single dataframe.

In [3]:
import geopandas
import pandas
import pathlib

In [26]:
df = pandas.DataFrame(data=[], columns=["State", "Democratic", "Republican", "Other", "Year"])

csv_folder = pathlib.Path("./raw_csv/")

for filepath in csv_folder.iterdir():
    if filepath.suffix == ".csv":
        year = filepath.stem
        
        table = pandas.read_csv(filepath)
        table["Year"] = year
        
        df = df.append(table)

To check our work, we verify that the `Year` column holds 8 unique values (one for each election cycle from 2002 to 2016):

In [29]:
df['Year'].describe()

count      419
unique       8
top       2016
freq        56
Name: Year, dtype: object

### Cleaning up NA values

Our table contains some empty rows, because not every state has a Senate election each year. We also have empty rows for territories without representation, like Puerto Rico and the District of Columbia.

We remove these empty records.

In [33]:
# Drop all records for elections that didn't happen
df = df.dropna(how="all", subset=("Democratic", "Republican", "Other"))

For some elections where a party received no votes (e.g. unopposed races) the party's cell was left empty. This is parsed as `NaN` by pandas. We correct these NA values to zeros.

In [35]:
df = df.fillna(0)

### Saving the dataset

Now that we have our desired dataset, we save it as `data.csv`.

In [37]:
df.to_csv("./data.csv", index=False) # omit the index, since it is meaningless

## Analysis